In [22]:
import torch 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
# 번역 전 문장 / 번역 후 문장 / 실제 번역 정답 문장
sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E'] 

In [5]:
# 세가지 문장을 임베딩하기 위해서 모든 문장 내 등장하는 단어 리스트와 해당 단어들의 빈도?를 나타내는 딕셔너리 생성
word_list = " ".join(sentences).split() 
word_list

['ich',
 'mochte',
 'ein',
 'bier',
 'P',
 'S',
 'i',
 'want',
 'a',
 'beer',
 'i',
 'want',
 'a',
 'beer',
 'E']

In [10]:
#set(word_list) # 중복제거
word_list = list(set(word_list))
word_list

['a', 'E', 'S', 'beer', 'bier', 'P', 'i', 'ich', 'ein', 'mochte', 'want']

In [12]:
word_dict = {w: i for i, w in enumerate(word_list)}
word_dict

{'E': 1,
 'P': 5,
 'S': 2,
 'a': 0,
 'beer': 3,
 'bier': 4,
 'ein': 8,
 'i': 6,
 'ich': 7,
 'mochte': 9,
 'want': 10}

In [14]:
number_dict = {i: w for i, w in enumerate(word_list)}
number_dict

{0: 'a',
 1: 'E',
 2: 'S',
 3: 'beer',
 4: 'bier',
 5: 'P',
 6: 'i',
 7: 'ich',
 8: 'ein',
 9: 'mochte',
 10: 'want'}

In [16]:
n_class = len(word_dict) # vocab list
n_class

11

In [26]:
n_hidden = 128
hidden = torch.zeros(1, 1, n_hidden)
hidden

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [27]:
# 앞서 만든 단어 빈도 딕셔너리를 기반으로 문장 임베딩
def make_batch():
    input_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[0].split()]]]
    output_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[1].split()]]]
    target_batch = [[word_dict[n] for n in sentences[2].split()]]

    # make tensor
    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

In [ ]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)

        # Linear for attention
        self.attn = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden * 2, n_class)